# Guidelines for Prompting
In this lesson, you'll practice two prompting principles and their related tactics in order to write effective prompts for large language models.

## Setup
#### Load the API key and relevant Python libaries.

In this course, we've provided some code that loads the OpenAI API key for you.

In [ ]:
!pip install litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import os
import litellm
from litellm import completion

# Set the OpenAI API key from the environment variables
litellm.openai_key = ''
litellm.api_base = ''

#### helper function
Throughout this course, we will use OpenAI's `qwen2.5` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat).

This helper function will make it easier to use prompts and look at the generated outputs:

In [ ]:
# Helper function to interact with a model
# We use the qwen 2.5 model and send a prompt in the format required for chat completions.

def get_completion(prompt, model="openai/qwen2.5", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = litellm.completion(
        model=model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [ ]:
# Example of using clear instructions in prompts to guide the model
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""

# Prompt asking the model to summarize the provided text into one sentence.
prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""

response = get_completion(prompt)
print(response)

To ensure the model produces the desired output, provide instructions that are clear, specific, and sufficiently detailed, even if it means the prompt is较长，但根据要求总结成一句话的话：

为了引导模型产生期望的输出，应提供清晰、具体且详尽的指令。


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [ ]:
# Example of asking for structured output in JSON format
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

Here is a JSON formatted response containing three made-up book titles along with their authors and genres:

```json
[
    {
        "book_id": 1,
        "title": "The Whispering Clocks",
        "author": "Lila Morningside",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "Echoes of the Forgotten Past",
        "author": "Ethan Vesper",
        "genre": "Historical Fiction"
    },
    {
        "book_id": 3,
        "title": "Quantum Shadows",
        "author": "Sophia Nexus",
        "genre": "Science Fiction"
    }
]
```


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [ ]:
# Example of prompting the model to format a sequence of steps if instructions are detected in the input text
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""

# Prompt asking the model to reformat the instructions in a stepwise manner if instructions are present
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let the tea sit for a few minutes so it can steep.
Step 5 - Take out the tea bag after a few minutes.
Step 6 - Add sugar or milk to taste if desired.


In [ ]:
# Example with text that does not contain instructions
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the \
trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. \
Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a \
perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""

# Same prompt structure to detect and format instructions
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting

In [ ]:
# Example of few-shot prompting: the model is given a style pattern to follow for its response
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

### User:

Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest valley flows from a modest spring; the grandest symphony originates from a single note; the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.


### Grandparent:
Resilience is like the bamboo that bends in the wind but never breaks. It’s the mountain that stands tall and unyielding through storm after storm. Just as a seed must push its way through hard soil to reach sunlight, you too will face challenges that test your limits. Each time you rise after falling, each step forward taken despite obstacles, strengthens your spirit and teaches you invaluable lessons. Embrace the storms for they shape the person you become.


### Principle 2: Give the model time to “think”

#### Tactic 1: Specify the steps required to complete a task

In [ ]:
# Example demonstrating task decomposition and formatting the output into JSON
text = f"""
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""

# Multi-step prompt asking the model to summarize, translate, and count names
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
In a village, siblings Jack and Jill joyfully climb a hill to fetch water but fall down after an accident, return home bruised but happy, and continue their adventures.

French translation:
Dans un village, les frères et sœurs Jack et Jill grimpent joyeusement une colline pour chercher de l'eau mais tombent après une malencontreuse chute, rentrent chez eux avec quelques bleus mais le moral intact et poursuivent leurs aventures avec délice.

Names in the French summary: Jack, Jill

{
  "french_summary": "Dans un village, les frères et sœurs Jack et Jill grimpent joyeusement une colline pour chercher de l'eau mais tombent après une malencontreuse chute, rentrent chez eux avec quelques bleus mais le moral intact et poursuivent leurs aventures avec délice.",
  "num_names": 2
}


#### Ask for output in a specified format

In [ ]:
# An alternative approach to specifying output format in detail
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill set out to fetch water from a well on a hill, but after tripping and tumbling down, they return home battered yet unbroken in spirit.

Translation: Jack et Jill se sont lancés dans une quête pour transporter de l'eau d'un puits sur une colline, mais après être tombés en glissant, ils sont rentrés chez eux, légèrement blessés mais leur esprit aventurier intact.

Names: ["Jack", "Jill"]

Output JSON: 
```json
{
  "french_summary": "Jack et Jill se sont lancés dans une quête pour transporter de l'eau d'un puits sur une colline, mais après être tombés en glissant, ils sont rentrés chez eux, légèrement blessés mais leur esprit aventurier intact.",
  "num_names": 2
}
```


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [ ]:
# Example demonstrating instructing the model to solve a problem before evaluating the student's solution
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is not entirely correct. The mistake lies in the calculation of maintenance cost, which should be $10 per square foot instead of $100 per square foot.

Here’s a corrected version:

Let \( x \) be the size of the installation in square feet.
Costs:
1. Land cost: \( 100x \)
2. Solar panel cost: \( 250x \)
3. Maintenance cost: \( 100,000 + 10x \)

Total cost for the first year of operations is:

\[ \text{Total Cost} = 100x + 250x + (100,000 + 10x) \]
\[ \text{Total Cost} = 360x + 100,000 \]

So the correct expression for the total cost in the first year as a function of \( x \) is:

\[ \boxed{360x + 100,000} \]


#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [ ]:
# Prompting the model to work through the problem independently before evaluating the student's solution
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000
```

Actual solution:
Let \( x \) be the size of the installation in square feet.

Costs:
1. Land cost: $100 per square foot.
   - This is directly proportional to \( x \), so the land cost is \( 100x \).

2. Solar panel cost: $250 per square foot.
   - This is also directly proportional to \( x \), so the solar panel cost is \( 250x \).

3. Maintenance cost:
   - There's a fixed yearly maintenance cost of $100,000.
   - Additionally, there’s an extra $10 per square foot, which means the variable part of the maintenance cost is \( 10x \).
   - Thus, the total maintenance cost is \( 100,000 + 10x \).

Adding these up:
\[ \text{Total Cost} = 100x + 250x + (100,000 + 10x) \]
Combining like terms:
\[ \text{Total Cost} = 360x + 100,000 \]

Is the student's solution the sam

## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [ ]:
# Example of the model having hallucination limitation
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is an innovative oral care product designed to provide users with a high-quality brushing experience, enhanced by smart technology features. Here are some key points about this toothbrush:

1. **Design and Comfort**: The AeroGlide is known for its ultra-thin bristles that fit comfortably between teeth and along the gum line without causing discomfort. Its sleek, compact design makes it easy to maneuver in hard-to-reach areas.

2. **Smart Technology**:
   - **Pressure Detection**: The toothbrush uses advanced pressure sensors to detect when you’re brushing too hard, which can help prevent damage to your gums.
   - **Brushing Time Guidance**: It guides users through a 2-minute brushing routine and breaks down the session into quadrants of the mouth for optimal cleaning.
   - **Connectivity**: Compatible with smartphones via an app, it allows users to track their brushing habits, receive personalized dental care recommendations, and monitor

## Try experimenting on your own!

#### Notes on using the OpenAI API outside of this classroom

To install the OpenAI Python library:
```
!pip install openai
```

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys).

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value:

```
import openai
openai.api_key = "sk-..."
```

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.